In [23]:
import pandas as pd
import requests

In [24]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [25]:



API_KEY = "8edf5a3557214b83a8d24412250109"
local = "Curitiba"  # pode ser "Curitiba, Brazil" ou latitude
url = "http://api.weatherapi.com/v1/forecast.json"
params = {
    "key": API_KEY,
    "q": local,
    "lang": "pt",     # opcional: traduz descrições para português
    "aqi": "no"       # opcional: inclui dados de qualidade do ar
}

try:
    response = requests.get(url, params=params)
    response.raise_for_status()
    dados = response.json()
    
    # Exemplo de como acessar alguns dados:
    # temperatura = dados["current"]["temp_c"]
    # descricao = dados["current"]["condition"]["text"]
    # umidade = dados["current"]["humidity"]
    # vento = dados["current"]["wind_kph"]

    # print(f"Tempo em {dados['location']['name']}: {descricao.capitalize()}")
    # print(f"Temperatura: {temperatura} °C")
    # print(f"Umidade: {umidade}%")
    # print(f"Vento: {vento} km/h")
   # print(dados)

except requests.exceptions.HTTPError as err:
    print(f"Erro HTTP: {err} — código: {response.status_code}")
except ValueError:
    print("Erro ao decodificar JSON.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

df = pd.json_normalize(dados)
df

    


,location.name,location.region,location.country,location.lat,location.lon,location.tz_id,location.localtime_epoch,location.localtime,current.last_updated_epoch,current.last_updated,current.temp_c,current.temp_f,current.is_day,current.condition.text,current.condition.icon,current.condition.code,current.wind_mph,current.wind_kph,current.wind_degree,current.wind_dir,current.pressure_mb,current.pressure_in,current.precip_mm,current.precip_in,current.humidity,current.cloud,current.feelslike_c,current.feelslike_f,current.windchill_c,current.windchill_f,current.heatindex_c,current.heatindex_f,current.dewpoint_c,current.dewpoint_f,current.vis_km,current.vis_miles,current.uv,current.gust_mph,current.gust_kph,forecast.forecastday
0,Curitiba,Parana,Brazil,-25.4167,-49.25,America/Sao_Paulo,1761506642,2025-10-26 16:24,1761506100,2025-10-26 16:15,22.3,72.1,1,Chuva fraca,//cdn.weatherapi.com/weather/64x64/day/296.png,1183,4.0,6.5,233,SW,1017.0,30.03,0.25,0.01,69,75,24.7,76.4,17.4,63.4,17.4,63.4,15.1,59.2,8.0,4.0,0.5,5.7,9.3,"[{'date': '2025-10-26', 'date_epoch': 17614368..."


In [26]:
forecast_list = dados["forecast"]["forecastday"]
df_forecast = pd.json_normalize(forecast_list)
df_forecast


,date,date_epoch,hour,day.maxtemp_c,day.maxtemp_f,day.mintemp_c,day.mintemp_f,day.avgtemp_c,day.avgtemp_f,day.maxwind_mph,day.maxwind_kph,day.totalprecip_mm,day.totalprecip_in,day.totalsnow_cm,day.avgvis_km,day.avgvis_miles,day.avghumidity,day.daily_will_it_rain,day.daily_chance_of_rain,day.daily_will_it_snow,day.daily_chance_of_snow,day.condition.text,day.condition.icon,day.condition.code,day.uv,astro.sunrise,astro.sunset,astro.moonrise,astro.moonset,astro.moon_phase,astro.moon_illumination,astro.is_moon_up,astro.is_sun_up
0,2025-10-26,1761436800,"[{'time_epoch': 1761447600, 'time': '2025-10-2...",20.4,68.8,15.6,60.0,17.8,64.0,5.6,9.0,2.32,0.09,0.0,8.0,4.0,73,1,87,0,0,Possibilidade de chuva irregular,//cdn.weatherapi.com/weather/64x64/day/176.png,1063,1.5,05:33 AM,06:29 PM,08:46 AM,11:14 PM,Waxing Crescent,17,0,0


In [27]:
# Extrair a lista de horas do primeiro dia de previsão
hour_list = df_forecast['hour'][0]  # Pega a lista de horas do primeiro dia

# Agora normalize essa lista para criar o dataframe de horas
df_hour = pd.json_normalize(hour_list)

# Converter para datetime
df_hour["time"] = pd.to_datetime(df_hour["time"])

# Definir como índice
df_hour = df_hour.set_index("time")

# Selecionar apenas colunas numéricas
df_numeric = df_hour.select_dtypes(include="number")

# Reamostrar de 5h em 5h e calcular média
df_5h = df_numeric.resample("5h").mean().reset_index()

print(df_5h.head())
df_5h

                 time    time_epoch  temp_c  temp_f  is_day  wind_mph  \
0 2025-10-26 00:00:00  1.761455e+09  17.160  62.860     0.0     3.040   
1 2025-10-26 05:00:00  1.761473e+09  18.620  65.500     0.8     3.360   
2 2025-10-26 10:00:00  1.761491e+09  19.920  67.880     1.0     2.800   
3 2025-10-26 15:00:00  1.761509e+09  18.060  64.520     0.8     3.080   
4 2025-10-26 20:00:00  1.761525e+09  15.775  60.375     0.0     2.975   

   wind_kph  wind_degree  pressure_mb  pressure_in  precip_mm  precip_in  \
0      4.92         32.2       1013.4      29.9240     0.0000      0.000   
1      5.40        211.6       1016.0      30.0040     0.0020      0.000   
2      4.54        222.6       1014.8      29.9720     0.0000      0.000   
3      4.98        234.4       1015.6      29.9880     0.4400      0.016   
4      4.80         43.0       1015.5      29.9825     0.0275      0.000   

   snow_cm  humidity  cloud  feelslike_c  feelslike_f  windchill_c  \
0      0.0     58.60   5.00       

,time,time_epoch,temp_c,temp_f,is_day,wind_mph,wind_kph,wind_degree,pressure_mb,pressure_in,precip_mm,precip_in,snow_cm,humidity,cloud,feelslike_c,feelslike_f,windchill_c,windchill_f,heatindex_c,heatindex_f,dewpoint_c,dewpoint_f,will_it_rain,chance_of_rain,will_it_snow,chance_of_snow,vis_km,vis_miles,gust_mph,gust_kph,uv,condition.code
0,2025-10-26 00:00:00,1.761455e+09,17.160,62.860,0.0,3.040,4.92,32.2,1013.4,29.9240,0.0000,0.000,0.0,58.60,5.00,17.160,62.860,17.160,62.860,17.160,62.860,8.980,48.14,0.00,0.0,0.0,0.0,10.0,6.0,6.380,10.280,0.00,1000.0
1,2025-10-26 05:00:00,1.761473e+09,18.620,65.500,0.8,3.360,5.40,211.6,1016.0,30.0040,0.0020,0.000,0.0,59.20,42.60,18.620,65.500,18.620,65.500,18.620,65.500,10.560,51.02,0.00,13.2,0.0,0.0,10.0,6.0,4.940,7.960,0.60,1013.8
2,2025-10-26 10:00:00,1.761491e+09,19.920,67.880,1.0,2.800,4.54,222.6,1014.8,29.9720,0.0000,0.000,0.0,65.20,38.80,19.920,67.880,19.920,67.880,19.920,67.880,13.120,55.64,0.00,0.0,0.0,0.0,10.0,6.0,3.820,6.140,5.68,1003.0
3,2025-10-26 15:00:00,1.761509e+09,18.060,64.520,0.8,3.080,4.98,234.4,1015.6,29.9880,0.4400,0.016,0.0,86.00,72.20,17.080,62.780,17.080,62.780,17.080,62.780,15.260,59.46,1.00,93.6,0.0,0.0,7.8,4.4,4.580,7.420,0.72,1128.4
4,2025-10-26 20:00:00,1.761525e+09,15.775,60.375,0.0,2.975,4.80,43.0,1015.5,29.9825,0.0275,0.000,0.0,95.25,64.75,15.775,60.375,15.775,60.375,15.775,60.375,15.025,59.05,0.25,25.0,0.0,0.0,2.0,1.0,5.275,8.475,0.00,1030.0


In [28]:
colunas_importantes = [
    "time",
    "feelslike_c",
    "humidity",
    "cloud",
    "uv",
    "wind_kph",
    "will_it_rain",
    "chance_of_rain",
]

df_limpo = df_5h[[c for c in colunas_importantes if c in df_5h.columns]]
df_limpo.sample(5)

,time,feelslike_c,humidity,cloud,uv,wind_kph,will_it_rain,chance_of_rain
2,2025-10-26 10:00:00,19.920,65.20,38.80,5.68,4.54,0.00,0.0
3,2025-10-26 15:00:00,17.080,86.00,72.20,0.72,4.98,1.00,93.6
4,2025-10-26 20:00:00,15.775,95.25,64.75,0.00,4.80,0.25,25.0
1,2025-10-26 05:00:00,18.620,59.20,42.60,0.60,5.40,0.00,13.2
0,2025-10-26 00:00:00,17.160,58.60,5.00,0.00,4.92,0.00,0.0
